In [7]:
import pandas as pd
import numpy as np
import re
import spacy

In [8]:
pd.set_option('display.max_colwidth', 1000)

In [9]:
all_amz_go = pd.read_csv('F:\\mailae_g\\ALL_MAG_2021+2022.csv', sep=';' )
all_cus_go = pd.read_csv('F:\\mailae_g\\ALL_PL_2022.csv', sep=';')

In [10]:
all_amz_go.sample(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
27666,156172,Delivery enquiry from Amazon customer Alina(Order: 203-8112983-6645102),"my order did not arrive ... Unfortunately I have to give bad review, because it turns out that I received, but unfortunately it did not .... and now I would like my money back please.",Email,freshdeskuser_21585880@channelreply.email,7 03 2022 01:02,3,2,"cr,rynek_uk,beautyshopok,beautyshopok_uk"
18894,297847,Aktualisierung durch den Verkäufer bezüglich Bestellnummer 306-4170409-6076369,"Hallo, leider kann ich kein Foto schicken, da der Artikel bereits heute morgen zurück geschickt wurde. Mit freundl. Grüßen R. Tönges",Email,freshdeskuser_24046115@channelreply.email,11 08 2022 20:49,1,1,"cr,rynek_de,konstinstall,konstinstall_de,FBM,rr"
6996,3008798,Package didn&#39;t arrive: Request a refund(Order: 302-0018870-3981119),"Ich sollte den Paket am 1 November haben , bis Heute habe ich gar nicht bekommen .",Email,freshdeskuser_25497902@channelreply.email,15 11 2022 16:42,2,2,"cr,rynek_de,sfd,sfd_de,FBM"
21846,235066,Frage von Amazon-Kunde Michael Schrefl(Bestellung: 306-9840722-6907546),"das Paket ist nicht angekommen und auch kein Schein, das das Paket bei der Post abzuholen wäre. bitte um Rückerstattung der Überweisung.",Email,freshdeskuser_23205803@channelreply.email,20 06 2022 10:54,4,2,"cr,rynek_de,aleeko,aleeko_de,FBM"
17892,309711,eBay Negative Feedback notification: #2078425090014. &quot;Türdrücker Türklinke Türbeschlag Türgriff Drückergarnitur PZ BB WC Langer Schild&quot;,You have received Negative feedback! ─ Message: ich habe falsch bestellt und direkt die Bestellung abgebrochen (storniert) aber der Verkäufer hat die Stornierung abgelehnt obwohl ich es 2 Minuten nachdem bestellen storniert hatte. Jetzt muss alles doppelt laufen ‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾ Comment Time: 2022-08-22 10:04:50 Item Title: Türdrücker Türk%se Türbeschlag Türgriff Drückergarnitur PZ BB WC Langer Schild Item Price: 5.17 Item ID: 192615620352 Transaction ID: 1954475458009 Order ID: 192615620352-1954475458009 BuyerID: casley0 Buyer&#39;s Rating Star: Blue ──────────────────────────── Feedback cannot be answered like normal message.,Email,freshdeskuser_24161897@channelreply.email,22 08 2022 12:45,1,1,"cr,ebay,rynek_de_eb,negativefeedback,kotarbau_de_eb,kotarbau"


In [11]:
all_cus_go.sample(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
46113,309689,[opóźnienie w dostawie z winy kuriera] 407-2735365-2365116 FR,"-- -- [image: eszybko] Daria Bielec Kierownik ds. sprzedaży i obsługi klienta ------------------------------ ul. Energetyków 5, 20-468 Lublin, Polska tel: +48 514 050 517 &lt;48514050517&gt;| e-mail: sklep@sportgrand.pl Treść tej wiadomości zawiera informacje poufne, przeznaczone tylko dla adresata. Udostępnianie, ujawnianie, powielanie, rozpowszechnianie bądź powoływanie się na jakikolwiek jej fragment przez inne osoby jest zabronione. W razie przypadkowego otrzymania tej wiadomości, prosimy o powiadomienie o tym nadawcy oraz trwałe jej usunięcie. This is a confidential e-mail intended solely for the use of the addressee(s). Unauthorized publication, use, dissemination or disclosure of this message, either in whole or in part is strictly prohibited. If you have received this message in error please send it back to the sender and delete it.",Email,sportstyle.gonito@gmail.com,22 08 2022 12:32,1,1,"Partner_email,Przychodzące_email,rynek_fr"
26082,364675,DE] 306-3104266-1904319 [FAKTURA],"Dzień dobry, klient prosi o fakturę do tego zamówienia 306-3104266-1904319. Pozdrawiam, Natalia",Outbound Email,shop@folnet.de,16 10 2022 18:32,1,1,rynek_de
34781,347638,[ANULACJA][FR] 402-1205899-6867528,"Dzień dobry, klient prosi o anulacje. Pozdrawiam, Anita",Outbound Email,gonito@fkwt.pl,27 09 2022 18:13,1,1,"Partner_wysłane,rynek_fr,Anulacja"
34527,287019,Re: Order cancelation request for Order ID: 302-5116226-9581951[*EMA/22/07/135435/95*],"Dzień dobry, Zwrot zlecony. Pozdrawiam serdecznie/Mit freundlichen Grüßen, Maciej Chamier Cieminski Specjalista ds. Obsługi Klienta tel: +48 58 746 37 97 w.41 E-Mail: m.cieminski@globalincome.pl Global Income s.c. ul.Chwaszczyńska 135B 81-571 Gdynia, Poland NIP: 586-216-73-15 Prosimy o zachowanie ciągłości korespondencji, bez zmiany tematu, gdyż zachowany jest w nim unikalny numer Państwa sprawy EMA/22/07/135435/95. Dziękujemy! ------ Hello, You have received an order cancelation request from a buyer. If you haven’t confirmed the shipment for this order, you may cancel it in Seller Central without impacting your cancelation rate. To cancel this order, click on the order ID below to be redirected to the Order Details page. On the Order Details page, select the reason for cancellation as Buyer Canceled, and cancel the order. Buyer Cancelation Request for Order ID: 302-5116226-9581951. If you have already shipped the order, the above link will not work. Please contact the buyer and ad...",Email,gonito@globalincome.pl,3 08 2022 14:01,0,1,"Partner_email,Przychodzące_email,Anulacja"
43952,308774,[REKLAMACJA] 028-5099646-7307525 DE,"Witam, pakiet dotarł uszkodzony, nie było folii w zestawie. Przesyła zdjęcia. Prosi o wymianę produktu i prosi by nie był to GLS. Pozdrawiam Joanna",Outbound Email,gonito@globalincome.pl,21 08 2022 20:19,1,1,"Partner_wysłane,rynek_de,Przypomnienie1,Przypomnienie2,Przypomnienie3,Pending6"


In [12]:
all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [13]:
# Funkcja do dzielenia komórek na listę, pierwszy argument tekst, drugi argument po czym tekst ma być dzielony 

def split_tags(tags: str, split_char: str) -> list:
    try:
        tags = tags.split(f'{split_char}')
        return tags 
    except Exception:
        return (list[tags])
    

In [14]:
# funkcja do wyciągania z listy, jakiego rynku dotyczą wiadomości

def get_market(lst: list) -> str:
    for i in lst:
        if "rynek" in i:
            return i

In [15]:
# funkcja do wyciągania numerów zamówień z tekstu i tytułu maila. Przyjmuje dwa różne ciągi tekstowe, sprawdza czy jest w tytule jest regex, jeśli nie to sprawdza czy jest w opisie, jeśli nie to zwraca pustą listę

se = re.compile(r"\d{3}-\d{7}-\d{7}")

def order_number(subject: str, description: str) -> list:
    subject = str(subject)
    description = str(description)
    if se.findall(subject) == []:
        return se.findall(description)
    return se.findall(subject)


In [16]:
# funkcja do tworzenia listy unikalnych wartości z kolumn. Dajemy wartość z kolumny i listę. Sprawdza czy wartość już znajduje się na liście, jeśli nie to ją dodaje

def all_order(order: list, new_list: list) -> list:
    try:
        if order[0] in new_list:
            return new_list
        else:
            new_list.append(order)
            return new_list
    except Exception:
        i =1

In [17]:
# druga funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

on = re.compile(r"\d{3}-\d{7}-\d{7}")
cc = re.compile(r"w{2}")
cc2 = re.compile(r'\[\w{2}\]')

def clear_title(title_list: list) -> list:
    for el in title_list:
        print(el)
    list_to_remove = []
    for i in range(len(title_list)):
        if re.match(on, title_list[i]):
            title_list.remove(i)
        elif re.match(cc, title_list[i]):
            title_list.remove(i)
        elif re.match(cc2, title_list[i]):
            title_list.remove(i)
    return title_list

In [18]:
# trzecia funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def num_there(s):
    return any(i.isdigit() for i in s)

def find_words_with_more_than_four_characters(words_with_tags):
  words = []
  for i in words_with_tags:
    if num_there(i):
      continue
    w = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', i)
    w = w.strip(" ")
    if len(w) >= 4:
      words.append(w)

  return words

In [19]:
# czwarta funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def find_words_with_more_than_four_characters2(subject: str) -> list:
  words = []
  new_subject = ''
  for l in subject:
    l = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', l)
    new_subject += l
  words_temp = new_subject.split(' ')
  for i in words_temp:
    if len(i) >= 4:
      words.append(i)

  return words

In [20]:
# Funkcja do zliczania elementów w liscie, zwraca listę unikatowych wartości i listę unikatowych wartości z informacją ile razy występowały

from collections import Counter

def count_occurrences(input_list):
    unique_values = set(input_list)
    occurrence_count = Counter(input_list)

    return list(unique_values), occurrence_count

In [21]:
# Funkcja sprawdza czy peirwszy elemnt tupli jest kluczem do słownika, jeśli tak to dodaje 0 elemnt do listy i zwraca set, który zawiera unikatowe wartości dla danego klucza

def find_all_values_to_dictonary (key: str ,list_of_tuple: list) -> set:
    di = []
    for i in list_of_tuple:
        if i[1] == key:
            di.append(i[0])
    return set(di)


In [22]:
# Zmiana tytułów na lowercase

all_amz_go['Subject'] = [str(row).lower() for row in all_amz_go['Subject']]
all_cus_go['Subject'] = [str(row).lower() for row in all_cus_go['Subject']]

In [23]:
# przekształcenie stringów w komórkach kolumny tags na listę, dzielone po przecinku 

all_amz_go['Tags'] = [split_tags(row, ',')for row in all_amz_go['Tags']]
all_cus_go['Tags'] = [split_tags(row, ',')for row in all_cus_go['Tags']]

In [24]:
# utworzenie kolumny, w której jest napisane jakiego rynku dotyczy wiadomość, na podstawie kolumny tags

all_amz_go['rynek'] = [get_market(row) for row in all_amz_go['Tags']]
all_cus_go['rynek'] = [get_market(row) for row in all_cus_go['Tags']]

In [25]:
# usunięcie wierszy, które w kolumnie 'rynek' nie maję 'rynek_de'

# all_amz_go.drop(all_amz_go[all_amz_go['rynek'] != 'rynek_de'].index,  inplace=True)
# all_cus_go.drop(all_cus_go[all_cus_go['rynek'] != 'rynek_de'].index,  inplace=True)

In [26]:
# wyciągnięcie numeru zamówień z kolumn subject i description

all_amz_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_amz_go['Subject'], all_amz_go['Description'])]
all_cus_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_cus_go['Subject'], all_cus_go['Description'])]

In [27]:
# resetowanie indexów

all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [28]:
# zmiana obiektów z kolummny 'Created Time' na daty

from datetime import datetime

format_date = "%d %m %Y %H:%M"

all_amz_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_amz_go['Created Time']] 
all_cus_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_cus_go['Created Time']] 

In [29]:
# wykorzystanie funkcji do splitowania aby utworzyć kolumnę 

all_amz_go['title_list'] = [split_tags(row, ' ') for row in all_amz_go['Subject']]
all_cus_go['title_list'] = [split_tags(row, ' ') for row in all_cus_go['Subject']]

In [30]:
# sprawdzenie czy funcja split_tags zadzaiała poprawnie

all_cus_go['title_list'][1]

['[de]', '302-0014409-7188317', 'niekompletne']

W tym momencie postanowieł sprawdzić, czy zamówienia, z jednej i drugiej tabeli się pokrywają w jakimś najmniejszym stopniu

In [31]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy jednego data set

all_amz_go_list = []
[all_order(row, all_amz_go_list) for row in all_amz_go['Order_number']]
all_amz_go_list = [item for sublist in all_amz_go_list for item in sublist]

In [32]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy drugiego data set

all_cus_go_list = []
[all_order(row, all_cus_go_list) for row in all_cus_go['Order_number']]
all_cus_go_list = [item for sublist in all_cus_go_list for item in sublist]

In [33]:
# zamieniam stworzyonych list na set

mcg_order_set = set(all_amz_go_list)
mag_rynek_de_order_set = set(all_cus_go_list)

In [34]:
# dodanie kontrolnie jednej takiej samej wartości do obu setów

mcg_order_set.add('123')
mag_rynek_de_order_set.add('123')

In [35]:
# porównanie dwóch

y = mcg_order_set.intersection(mag_rynek_de_order_set)

In [36]:
# Wyświetlenie ile mamy takich samych numerów zamówień w jednym i drugim secie

len(y)

11037

Powrót do preprocesingu danych

In [37]:
# weryfickaja działania funkcji do czyszczenia tytułów maili

print(find_words_with_more_than_four_characters2(all_cus_go['Subject'][5552]))

['zapytanie', 'amandine', 'perez']


In [38]:
# utworzenie nowej kolumny label i dodanie do niej słów, które nadają się na labelki

all_cus_go['label'] = [find_words_with_more_than_four_characters2(row) for row in all_cus_go['Subject']]

In [39]:
# sprawdzenie jak wygląda labels dla 10 przykładowych wierszy

all_cus_go['label'].sample(10)

14791    [case, other, account, issues]
50597                      [reklamacja]
19295                           [zwrot]
34006         [odbiór, osobisty, pilne]
26544                         [dostawa]
12993                      [anulowanie]
43819                         [pytanei]
25989                      [reklamacja]
49185                       [niedziała]
19165                                []
Name: label, dtype: object

In [40]:
# Stworzenie listy ze wszystkimi wartościami z kolumny labels

label_list = []
[all_order(row, label_list) for row in all_cus_go['label']]
label_list = [item for sublist in label_list for item in sublist]

In [41]:
# Stworzenie dwóch list na podstawie listy wartości z kolumny['label'], jedna lista to unikatowe wartości, druga lista unikatowe wartości z informacją ile dana wartość występowała

uniq_vales, uniq_values_count = count_occurrences(label_list)

In [42]:
# Stworzenie obiektu spacy z polskim słownikiem

nlp = spacy.load("pl_core_news_md")

In [43]:
# Zmiana unikatowej listy na string

uniq_vales_sentence = ' '.join(uniq_vales)

In [44]:
# tokenizacja stringu unikatowych wartości z kolumny tags

doc = nlp(uniq_vales_sentence)

In [45]:
# lemantyzacja wcześniej stworzonych tokenów. Stworzenie listy i listy tupli. Lista tupli zawierające słowo i jej podstawę słowotwórczą, sama lista zawiera wszystkie podstwy słowotwórcze 

all_lem = []
all_words = []
for token in doc:
    all_words.append((token.text, token.lemma_))
    all_lem.append(token.lemma_)

In [46]:
# import pickle
# with open('all_lem.pkl', 'wb') as file:
#     pickle.dump(all_lem, file)

In [47]:
# Stworzenie słownika gdzie klucz to podstawa słowotwórcza a values jest listą słów dla tej podstawy występująca w kolumnie tags

lem_dict = {}
for i in set(all_lem):
    lem_dict[i] = find_all_values_to_dictonary(i,all_words)

In [48]:
# stworzenie słownika ze specjalnego obieku counter

uniq_values_count_dict = dict(uniq_values_count)

In [49]:
# Stworzenie słownika, gdzie klucz to podstawa słwowtwórcza, a values to suma wartości występowania słów zawiązanych z tą lemą

number_of_occurence = {}
for i in lem_dict:
    number_of_occurence_value = []
    for j in lem_dict[i]:
        number_of_occurence_value.append(uniq_values_count_dict[j])
    number_of_occurence[i] = sum(number_of_occurence_value)

In [50]:
# Posortowanie słownika zawierającego podstawę słowotwórczą oraz ilość wystąpnień słów z nim powiązanych

number_of_occurence_sorted = sorted(number_of_occurence.items(), key=lambda x:x[1], reverse=True)

In [51]:
list(lem_dict)

['ocde',
 'alois',
 'gerung',
 'parcel',
 'cwke',
 'bbdf',
 'qlccb',
 'myhegk',
 'miranda',
 'qbef',
 'sbjj',
 'damian',
 'vhfrstsg',
 'forever',
 'nastyvia',
 'store',
 'jung',
 'cgwwjwq',
 'giannuzzi',
 'mlsf',
 'waldsch',
 'romański',
 'opóźniić',
 'reklamac',
 'Mikesch',
 'regnira',
 'stato',
 'whsra',
 'terramax',
 'franke',
 'calmon',
 'powiadomić',
 'protokół',
 'usuniecie',
 'tools',
 'antonć',
 'requested',
 'mdyqua',
 'kurier',
 'jpzc',
 'anita',
 'join',
 'banasiewicz',
 'rtmky',
 'nowakowski',
 'kupujacy',
 'dbpyjcmb',
 'clare',
 'olsson',
 'npvj',
 'uhpa',
 'qvwq',
 'mbhb',
 'mfug',
 'anulwoanie',
 'Lwvtq',
 'gual',
 'kqlg',
 'struppy',
 'qvpcjn',
 'Andrea',
 'ammirata',
 'wolny',
 'annalisa',
 'rispondere',
 'vczn',
 'Alisei',
 'ysszrw',
 'przejecie',
 'kaylan',
 'hendrick',
 'rechnungen',
 'matthi',
 'miriać',
 'undelievered',
 'waldemar',
 'rekamacja',
 'specmark',
 'nideostarczone',
 'książka',
 'vivien',
 'colalight',
 'unstimmigkeit',
 'whkw',
 'wfdrx',
 'soke',
 'li

In [52]:
type(number_of_occurence_sorted)

list

In [53]:
df_number_of_occurence_sorted = pd.DataFrame(number_of_occurence_sorted, columns=['lema', 'ilość wystapień'])

In [54]:
df_number_of_occurence_sorted.head(5)

,lema,ilość wystapień
0,zwrot,6128
1,reklamacja,4539
2,dostawa,3243
3,opóźnienie,2947
4,niedostarczona,2812


In [55]:
df_number_of_occurence_sorted.to_csv('number_of_occurence_sorted.csv', sep=',', encoding='cp1250',index=False)

In [56]:
labels_df = pd.read_csv('labels.csv', sep=';', header=None)
labels_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,anulacja,anulować,anulacja,anulowac,anualcja,anulacj,anuowanie,anulownie,nieanulować,anulwoanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,reklamacja,reklamacja,reklamcja,reklaamcja,reklamajca,rekalamacjo,rekalmacja,reklmacja,rekalamcja,reklamac,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wysyłka,wysyłce,wysyłka,wysłać,wysłek,wysyłke,wysyka,wysylce,wysyłką,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,refund,refund,refundacja,refunda,refunf,refung,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,przesyłka,przesyłka,przesylka,przesyłke,przesyłewka,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,dostawa,dostawa,niedostarczć,niedostarczić,niedostarczono,dostarczyć,dostarczć,niedostraczć,niedostraczone,dostarczenie,...,dostawty,niedostacozne,niedostarczoony,nniedostarczon,dostwa,dostepność,niedostarczpme,niedostarczonea,dostaroczić,niedostarczone
6,opóźnienie,opóźniić,opóźnienie,opóźniyć,opoznienie,opoźnieć,opóźnieniu,opóżniona,opóxnienie,opoźnienia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,uszkodzony,uszkodzić,uszkodzona,uszkodzeć,uszkodozny,uszkdzoć,uszkodzonie,uszkodzdenie,uszkodczć,uszkodzć,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,adres,adres,adresse,adresowy,adresskl,zaadresować,adresie,adrres,padres,adress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ponowna,ponowny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# etykiety z wysortowanymi słowami kluczowymi

niedostarczone = 'adres	adresse	adresowy	adresskl	zaadresować	adresie	adrres	padres	adress	adre',																																																	
anulacja =	'anulować	anulacja	anulowac	anualcja	anulacj	anuowanie	anulownie	nieanulować	anulwoanie	anualacja	anuelowanie	anulownać	anuloewanie	nieanulowanie	anulownaie	anulaj	anuloanie',																																										
reklamacja = 'reklamacja	reklamcja	reklaamcja	reklamajca	rekalamacjo	rekalmacja	reklmacja	rekalamcja	reklamac	reklama	reklamancja	reklamować	reklaamacja	reklamcaja	rekamacja	reklamaja	reklmamacja',																																										
wysyłka	= 'wysyłce	wysyłka	wysłać	wysłek	wysyłke	wysyka	wysylce	wysyłką',																																																			
refund = 'refund	refundacja	refunda	refunf	refung',																																																						
przesyłka =	'przesyłka	przesylka	przesyłke	przesyłewka',																																																							
dostawa = 'dostawa	niedostarczć	niedostarczić	niedostarczono	dostarczyć	dostarczć	niedostraczć	niedostraczone	dostarczenie	dostarczeć	dostępność	niedostaczone	dostęp	niedostarczony	niedostraczony	niedostarcone	niedostarczny	niedostaczon	niedostarcozny	dostepnosc	nieodostraczone	niedostrczone	niedostaraczyć	dostawca	niedostarzona	nedostarczona	niedostaraczone	niiedostarczona	niedostarczine	niedostrczona	dostway	nedostarczone	dostaczić	dostrarczona	dostepnnosc	dostrczić	niedostarczeć	dostawe	nidostarczone	niedostaroczone	niedostaeczone	ninedostarczony	niodostarczony	niedostyarczić	niedostarczoana	dostraczć	dostarcoyno	niedostarzone	niedostarczzona	niesdostarczone	dostawty	niedostacozne	niedostarczoony	nniedostarczon	dostwa	dostepność	niedostarczpme	niedostarczonea	dostaroczić',
opóźnienie = 'opóźniić	opóźnienie	opóźniyć	opoznienie	opoźnieć	opóźnieniu	opóżniona	opóxnienie	opoźnienia	opóźnienieć	opoźnenie	opoznić	opóźenienie	opóżnienie	opoznieć',																																												
uszkodzony = 'uszkodzić	uszkodzona	uszkodzeć	uszkodozny	uszkdzoć	uszkodzonie	uszkodzdenie	uszkodczć	uszkodzć	uszkopdzenie	uszkodzaić',																																															
adres =	'adres	adresse	adresowy	adresskl	zaadresować	adresie	adrres	padres	adress	adre',																																																	
ponowna = 'ponowny',																																																										
paczka = 'paczka	paczkomat	paczk	paczkmać',																																																							
zapytanie =	'zapytanie	zapyatnie	zapyatanie	zapyatenie	zapytanienie	zapytamie	zaptanie	zapatynie	zapytatć	zaptytanie	zapypytanie	zapytane	zpytanąć',																																														
zwrot =	'zwrot	zwrotny	zwrtot	zwrto	zwrócić	zzwrot',																																																					
pytanie = 'pytanie	pytanei	pytnaie	pytnie',																																																							
brak = 'brak	brakować	brakujący	brakujyć',																																																							
niekompletne = 'niekompletny',																																																										
faktura = 'faktura	fattura	fakura',																																																								
inne =	'inny	innen'																																																								


In [58]:
labels = [niedostarczone,anulacja,reklamacja,wysyłka,refund,przesyłka,dostawa,opóźnienie,uszkodzony,adres,ponowna,paczka,zapytanie,zwrot,pytanie,brak,niekompletne,faktura,inne]

In [59]:
labels_df = labels_df.T
labels_df.columns = labels_df.iloc[0]

In [60]:
labels_df.sample(3)

,anulacja,reklamacja,wysyłka,refund,przesyłka,dostawa,opóźnienie,uszkodzony,adres,ponowna,paczka,zapytanie,zwrot,pytanie,brak,niekompletne,faktura,inne
22,NaN,NaN,NaN,NaN,NaN,niedostrczone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,nieanulowanie,reklamcaja,NaN,NaN,NaN,niedostarczony,opóżnienie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,anulacja,reklamcja,wysyłka,refundacja,przesylka,niedostarczć,opóźnienie,uszkodzona,adresse,NaN,paczkomat,zapyatnie,zwrotny,pytanei,brakować,NaN,fattura,innen


In [61]:
labels_dict = labels_df.to_dict(orient='list')

In [62]:
print(labels_dict['anulacja'])

['anulacja', 'anulować', 'anulacja', 'anulowac', 'anualcja', 'anulacj', 'anuowanie', 'anulownie', 'nieanulować', 'anulwoanie', 'anualacja', 'anuelowanie', 'anulownać', 'anuloewanie', 'nieanulowanie', 'anulownaie', 'anulaj', 'anuloanie', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [63]:
test1 = {'test1' : "['anulacja', 'anulować', 'anulacja', 'anulowac', 'anualcja', 'anulacj', 'anuowanie', 'anulownie', 'nieanulować', 'anulwoanie', 'anualacja', 'anuelowanie', 'anulownać', 'anuloewanie', 'nieanulowanie', 'anulownaie', 'anulaj', 'anuloanie', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]",}

In [64]:
test1
key_lvl1_test = []
key_lvl1_test.append(test1.keys())
print([print(i) for i in key_lvl1_test])

dict_keys(['test1'])
[None]


In [65]:
def get_all_values (dic1: dict) -> dict:
    key_lvl1 = []
    key_lvl1.append(dict.keys)
    for i in ke

SyntaxError: expected ':' (2859337441.py, line 4)

In [ ]:
test1 BBBB